In [34]:
import json
import pickle
from collections import Counter
from pprint import pprint

In [2]:
crowd_dir = '/disk_4/KFN/crowdsourcing/'
M = ['M1', 'M2', 'M3', 'M4', 'M5']

In [68]:
def load_data():
    alldata, data = [],[]
    for m in M:
        fname = crowd_dir+m+'_data.json'
        with open(fname, 'r') as f:
            d = json.load(f)
            alldata += d
    data = {}
    for d in alldata:
        if 'prop' in d['id'] or 'sejong' in d['id']:
            tokens = d['tokens']
            lu = d['lu']
            lus = ['_' for i in range(len(tokens))]
            args = ['_' for i in range(len(tokens))]
            for tid in d['target']:
                lus[tid] = lu
            for arg in d['arguments']:
                arg_id = arg['arg_id']
                n = 0
                for tok_id in arg['tokens']:
                    args[tok_id] = str(arg_id)+'-'+str(n)
                    n += 1
            sent = []
            sent.append(tokens)
            sent.append(lus)
            sent.append(args)
            data[d['sent_id']] = sent
    return data
    
data = load_data()

In [130]:
crowdsourcing_data = []
for m in M:
    fname = crowd_dir+m+'_data.json'
    with open(fname, 'r') as f:
        m_data = json.load(f)
        crowdsourcing_data += m_data
print(len(crowdsourcing_data))

10000


In [132]:
def sid2source(sid):    
    for d in crowdsourcing_data:
        if sid == d['sent_id']:
            source = d['source']
            break
    return source
sid2source(10062)

'propbank'

In [54]:
def gen_crowd_data():
    crowd_data = {}
    for m in M:
        eval_file = crowd_dir+m+'_precessed_results.pickle'
        with open(eval_file, 'rb') as f:
            processed_results = pickle.load(f)
        
        sent_list = list(processed_results["units"].index)
        print(m)
        for i in range(len(sent_list)):
            sent_id = sent_list[i]
            fss = processed_results["units"]["unit_annotation_score"][sent_id]
            pred_by_crowd = fss.most_common(1)[0][0]
            score = fss.most_common(1)[0][1]
            ftuple = (pred_by_crowd, score)
            crowd_data[sent_id] = ftuple
    return crowd_data
            
crowd_frame= gen_crowd_data()

M1
M2
M3
M4
M5


In [61]:
def load_crowd_data():
    crowd_data = {}
    for m in M:
        crowd_file = crowd_dir+m+'_result.json'
        with open(crowd_file) as f:
            crowd = json.load(f)
        for sid in crowd:
            annotations = crowd[sid]
            frame_dict = {}
            for anno in annotations:
                frame = anno['checked_frame_candidates']
                if anno['worker'] != 'kaist_framenet@crowdworks.kr':
                    if frame not in frame_dict:
                        frame_dict[frame] = anno['aguments']
                    else:
                        args = frame_dict[frame]
                        args += anno['aguments']
                        frame_dict[frame] = args
            for frame in frame_dict:
                all_args = frame_dict[frame]
                arg_dict = {}
                arg_list = []
                for arg in all_args:
                    arg_list.append(arg['arg_id'])
                arg_list = list(set(arg_list))
                for arg_id in arg_list:
                    args = []
                    for arg in all_args:
                        if arg_id == arg['arg_id']:
                            args.append(arg['tag'])
                    arg_dict[arg_id] = Counter(args)
                frame_dict[frame] = arg_dict
                
            sid = int(sid)
            if sid not in crowd_data:
                crowd_data[sid] = frame_dict
    return crowd_data
crowd_data = load_crowd_data()

In [73]:
print(len(data))
print(data[0])

9500
[['인도네시아', '환경당국은', '수마트라섬', '중앙부에서', '짙은', '연기가', '급격히', '퍼지고', '있으며', '2', '주전', '우기가', '끝나면서', '증가하기', '시작한', '산불발생', '지역도', '지난달', '28', '일의', '80', '곳에서', '지난', '3', '일에는', '165', '곳으로', '급증했다고', '밝혔다.'], ['_', '_', '_', '_', '_', '_', '_', '퍼지다.v', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['0-0', '0-1', '1-0', '1-1', '2-0', '2-1', '3-0', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']]


In [58]:
print(crowd_frame[0])

('Dispersal', 0.617163446529057)


In [62]:
pprint(crowd_data[0])

{'Dispersal': {0: Counter({'none': 4, 'Individuals': 1, 'Agent': 1}),
               1: Counter({'Place': 5, 'Agent': 1}),
               2: Counter({'Individuals': 3, 'Cause': 2, 'none': 1}),
               3: Counter({'Manner': 5, 'none': 1})},
 'Expansion': {0: Counter({'none': 1}),
               1: Counter({'Place': 1}),
               2: Counter({'Item': 1}),
               3: Counter({'Manner': 1})},
 'Filling': {0: Counter({'none': 2}),
             1: Counter({'Place': 2}),
             2: Counter({'Theme': 2}),
             3: Counter({'Manner': 2})},
 'none': {0: Counter({'': 1}),
          1: Counter({'': 1}),
          2: Counter({'': 1}),
          3: Counter({'': 1})}}


In [79]:
def gen_kfn11_crowd_data(threshold):
    kfn11 = {}
    for sid in data:
        old_conll = data[sid]
        frame, score = crowd_frame[sid]
        
        if score >= threshold:
            if frame != 'none':
        
                crowd = crowd_data[sid]
                tokens = old_conll[0]
                lus = old_conll[1]
                old_args = old_conll[2]

                frames = ['_' for i in range(len(lus))]
                for idx in range(len(lus)):
                    lu = lus[idx]
                    if lu != '_':
                        frames[idx] = frame

                args = ['O' for i in range(len(lus))]
                for argidx in range(len(old_args)):
                    old_arg = old_args[argidx]
                    if old_arg != '_':
                        arg_id = int(old_arg.split('-')[0])
                        if old_arg.split('-')[1] == '0':
                            bio = 'B-'
                        else:
                            bio = 'I-'
                        if old_arg != '_':
                            fe = crowd[frame][arg_id].most_common(1)[0][0]
                            if fe != 'none':
                                args[argidx] = bio+fe
                            
                for argidx in range(len(args)):
                    if argidx < len(args):
                        current_tag = args[argidx]
                        next_tag = args[argidx]
                        
                conll = []
                conll.append(tokens)
                conll.append(lus)
                conll.append(frames)
                conll.append(args)
                            
                if sid not in kfn11:
                    kfn11[sid] = conll

    if threshold == 0:
        fname = '../data/1.1/crowdsourcing/all.json'
    elif threshold == 0.5:
        fname = '../data/1.1/crowdsourcing/bestF1.json'
    elif threshold == 0.9:
        fname = '../data/1.1/crowdsourcing/correct.json'
    
    with open(fname, 'w') as f:
        json.dump(kfn11, f, ensure_ascii=False, indent=4)
        
    return kfn11

kfn_a = gen_kfn11_crowd_data(0)
kfn_b = gen_kfn11_crowd_data(0.5)
kfn_c = gen_kfn11_crowd_data(0.9)

In [154]:
def load_kfn10():
    with open('../resource/1.0/KFN_annotations.json','r') as f:
        kfn10 = json.load(f)
    return kfn10
kfn10 = load_kfn10()

In [119]:
for i in kfn10:
    print(i)
    print(kfn10[i])
    break

0
{'lu': '이익.n.Earnings_and_losses', 'frame': 'Earnings_and_losses', 'denotations': [{'id': 1, 'obj': 'Earnings_and_losses', 'span': {'begin': 81, 'end': 85}, 'token_span': [18], 'role': 'TARGET', 'text': '순이익이'}, {'id': 2, 'obj': 'Earner', 'token_span': [12, 13, 14, 15, 16], 'span': {'begin': 48, 'end': 76}, 'text': 'Aetna Life and Casualty Co.의', 'role': 'ARGUMENT'}, {'id': 3, 'obj': 'Time', 'token_span': [17], 'span': {'begin': 77, 'end': 80}, 'text': '3분기', 'role': 'ARGUMENT'}], 'relations': [{'subj': 1, 'obj': 3, 'pred': 'arg'}, {'subj': 1, 'obj': 3, 'pred': 'arg'}], 'text': '태풍 Hugo가 남긴 피해들과 회사 내 몇몇 주요 부서들의 저조한 실적들을 반영하여, Aetna Life and Casualty Co.의 3분기 순이익이 182.6 백만 달러 또는 주당 1.63 달러로 22 % 하락하였다.', 'tokens': ['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '순이익이', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], 'sent_id': 'en.FrameNet-1', 'source': 'en.FrameNet'}


17434
17434


In [136]:
def get_token_span(token_ids, text):
    text = text.split(' ')
    d = {}
    cid = -1
    for i in range(len(text)):
        if i == 0:
            d[i] = (0, len(text[i]))
            cid = len(text[i]) +1
        else:
            d[i] = (cid, cid+len(text[i]))
            cid =  cid+len(text[i]) +1
    start = True
    for i in token_ids:
        if start == True:
            begin = d[i][0]
            start = False
        end = d[i][1]
    span = {}
    span['begin'] = begin
    span['end'] = end
    return span

text = '나는 밥을 먹었다'
span = get_token_span([1,2], text)
print(span)

def gen_denotations(tokens, text, frame_index):
    # 1) find target
    target_ids = []
    for token in tokens:
        if token[2] != '_':
            target_ids.append(int(token[0]))
            target = token[2]
    span = get_token_span(target_ids, text)
    deno = {}
    deno['id'] = 1
    deno['obj'] = frame_index
    deno['span'] = span
    deno['token_span'] = target_ids
    deno['role'] = 'TARGET'
    b = span['begin']
    e = span['end']
    deno['text'] = text[b:e]
    denos = []
    denos.append(deno)
    # 2) find arguments
    fe_id = 2
    for i in range(len(tokens)):
        if tokens[i][4] != 'O':
            fe_bio = tokens[i][4]
            fe_list = fe_bio.split('-')[1:]
            fe = '_'.join(fe_list)
            if fe_bio.startswith('B'):
                d = {}
                d['id'] = fe_id
                d['obj'] = fe.title()                
                n = 1
                arg_span = []
                arg_span.append(int(tokens[i][0]))
                nextfe = False
                while i+n < len(tokens):
                    nextfe = tokens[i+n][4]
                    if nextfe == 'I-'+fe:
                        arg_span.append(int(tokens[i+n][0]))
                        n += 1
                    else:
                        break                    
                fe_id += 1
                d['token_span'] = arg_span
                span = get_token_span(arg_span, text)
                d['span'] = span
                b = span['begin']
                e = span['end']
                d['text'] = text[b:e]
                d['role'] = 'ARGUMENT'
                denos.append(d)
            else:
                pass
    if denos:
        return denos
    else:
        print('ERROR_gen_denotations')
        
def get_relations(denos):
    relation = {}
    relation['subj'] = 1
    relations = []
    for i in denos:
        if i['role'] == 'ARGUMENT':
            did = i['id']
            relation['obj'] = did
            relation['pred'] = 'arg'
            relations.append(relation)
    return relations

def get_span_lu_frame(conll):
    span = []
    for idx in range(len(conll)):
        i = conll[idx]
        if i[2] != '_':
            lu = i[2]
            frame = i[3]
            span.append(idx)
    lu = lu+'.'+frame
    return span, lu, frame

{'begin': 3, 'end': 9}


In [149]:
h = kfn10['1']
print(h)

{'lu': '분기.n.Calendric_unit', 'frame': 'Calendric_unit', 'denotations': [{'id': 1, 'obj': 'Calendric_unit', 'span': {'begin': 77, 'end': 80}, 'token_span': [17], 'role': 'TARGET', 'text': '3분기'}, {'id': 2, 'obj': 'Relative_Time', 'token_span': [17], 'span': {'begin': 77, 'end': 80}, 'text': '3분기', 'role': 'ARGUMENT'}], 'relations': [{'subj': 1, 'obj': 2, 'pred': 'arg'}], 'text': '태풍 Hugo가 남긴 피해들과 회사 내 몇몇 주요 부서들의 저조한 실적들을 반영하여, Aetna Life and Casualty Co.의 3분기 순이익이 182.6 백만 달러 또는 주당 1.63 달러로 22 % 하락하였다.', 'tokens': ['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '순이익이', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], 'sent_id': 'en.FrameNet-1', 'source': 'en.FrameNet'}


In [157]:
def gen_kfn11_anno_file():
    anno_id = 17437
    kfn10 = load_kfn10()
    print('kfn10', len(kfn10))
    kfn11 = kfn10
    for sid in kfn_b:
        d = kfn_b[sid]
        tokens = d[0]
        lus = d[1]
        frames = d[2]
        args = d[3]     
#         for idx in range(len(tokens)):
#             if lus[idx] != '_':
#                 lu = lus[idx]
#                 frame = frames[idx]
        text = ' '.join(tokens)
        sent_id = 'crowdsourcing-'+str(sid)
        source = sid2source(int(sid))
        
        tokids = []
        for tokid in range(len(tokens)):
            tokids.append(str(tokid))
            
        conll = []
        for tokid in range(len(tokens)):
            t = []
            t.append(tokids[tokid])
            t.append(tokens[tokid])
            t.append(lus[tokid])
            t.append(frames[tokid])
            t.append(args[tokid])
            conll.append(t)
        
        tgt_span, lu, frame = get_span_lu_frame(conll)
        denos = gen_denotations(conll, text, frame)
        relations = get_relations(denos)
        
        item = {}
        item['lu'] = lu
        item['frame'] = frame
        item['denotations'] = denos
        item['relations'] = relations
        item['text'] = text
        item['tokens'] = tokens
        item['sent_id'] = sent_id
        item['source'] = source
        
        annotation_id = str(anno_id)
        anno_id +=1
        if annotation_id not in kfn11:
            kfn11[annotation_id] = item
        else:
            print(annotation_id)
            
    print('kfn11',len(kfn11))
    
    
    return kfn11
    
kfn11 = gen_kfn11_anno_file()


kfn10 17434
kfn11 25498


In [161]:
old_kfn11 = kfn11
new_kfn11 = {}

for i in old_kfn11:
    item = old_kfn11[i]
    
    ok = ['육개.n.Cardinal_numbers', '년.n.Cardinal_numbers', '모두.n.Cardinal_numbers', '사백.n.Cardinal_numbers', '삼백.n.Cardinal_numbers', '영하.n.Cardinal_numbers', '육십.n.Cardinal_numbers', '이틀.n.Cardinal_numbers', '일.n.Cardinal_numbers', '일련.n.Cardinal_numbers', '하루.n.Cardinal_numbers']
    
    if item['frame'] == 'Cardinal_numbers':
        if item['lu'] in ok:
            new_kfn11[i] = item
        else:
            pass
        
    else:
        new_kfn11[i] = item
        
print('new_kfn11', len(new_kfn11))
    
        
    

with open('../resource/1.1/KFN_annotations.json','w') as f:
    json.dump(new_kfn11, f, ensure_ascii=False, indent=4)    
print('written is done')

new_kfn11 25483
written is done


In [170]:
def kfn2data():
    with open('../resource/1.1/KFN_annotations.json','r') as f:
        kfn11 = json.load(f)
        
    print('kfn11', len(kfn11))
       
    
    kfn11_data = {}
    for anno_id in kfn11:

        lemma = kfn11[anno_id]['lu'].split('.')[0]
        pos = kfn11[anno_id]['lu'].split('.')[1]
        
        lu = lemma+'.'+pos
        frame = kfn11[anno_id]['frame']
        tokens = kfn11[anno_id]['tokens']
        lus = ['_' for i in range(len(tokens))]
        frames = ['_' for i in range(len(tokens))]
        args = ['O' for i in range(len(tokens))]
        for deno in kfn11[anno_id]['denotations']:
            if deno['role'] == 'TARGET':
                for tokid in deno['token_span']:
                    lus[tokid] = lu
                    frames[tokid] = frame
        for deno in kfn11[anno_id]['denotations']:
            if deno['role'] == 'ARGUMENT':
                for aidx in range(len(deno['token_span'])):
                    tokid = deno['token_span'][aidx]
                    if aidx == 0:
                        bio = 'B-'
                    else:
                        bio = 'I-'
                    fe = bio+deno['obj']
                    args[tokid] = fe
        tokid = 0
        tokids = []
        for i in range(len(tokens)):
            tokids.append(str(tokid))
            tokid +=1
        
        conll = []
        for tokid in range(len(tokens)):
            t = []
            t.append(tokids[tokid])
            t.append(tokens[tokid])
            t.append(lus[tokid])
            t.append(frames[tokid])
            t.append(args[tokid])
            conll.append(t)
        
        kfn11_data[anno_id] = conll
        
    print('kfn11', len(kfn11_data))
        
    return kfn11_data
kfn11_data = kfn2data()


kfn11 25483
kfn11 25483


In [183]:
import random
all_sents = list(kfn11_data.keys())

tst = random.sample(all_sents, 5097)
sents = [i for i in all_sents if not i in tst]
dev = random.sample(sents, 2548)
trn = [i for i in sents if not i in dev]


print('all', len(all_sents))
print('tst',len(tst))
print('dev', len(dev))
print('trn', len(trn))


tst_data, dev_data, trn_data = {},{},{}

for i in kfn11_data:
    item = kfn11_data[i]
    if i in tst:
        tst_data[i] = item
    elif i in dev:
        dev_data[i] = item
    elif i in trn:
        trn_data[i] = item
        
print('tst',len(tst_data))
print('dev', len(dev_data))
print('trn', len(trn_data)) 

all 25483
tst 5097
dev 2548
trn 17838
tst 5097
dev 2548
trn 17838


In [188]:
with open('../data/1.1/training.tsv','w') as f:
    for i in trn_data:
        item = trn_data[i]
        
        text_list = []
        for t in item:
            text_list.append(t[1])
        text = ' '.join(text_list)
        
        line = '#anno-id:'+str(i)+'\n'
        f.write(line)
        
        line = '#text:'+text+'\n'
        f.write(line)
        
        for i in item:
            line = '\t'.join(i)+'\n'
            f.write(line)
        
        line = '\n'
        f.write(line)
        
with open('../data/1.1/test.tsv','w') as f:
    for i in tst_data:
        item = tst_data[i]
        
        text_list = []
        for t in item:
            text_list.append(t[1])
        text = ' '.join(text_list)
        
        line = '#anno-id:'+str(i)+'\n'
        f.write(line)
        
        line = '#text:'+text+'\n'
        f.write(line)
        
        for i in item:
            line = '\t'.join(i)+'\n'
            f.write(line)
        
        line = '\n'
        f.write(line)
        
with open('../data/1.1/dev.tsv','w') as f:
    for i in dev_data:
        item = dev_data[i]
        
        text_list = []
        for t in item:
            text_list.append(t[1])
        text = ' '.join(text_list)
        
        line = '#anno-id:'+str(i)+'\n'
        f.write(line)
        
        line = '#text:'+text+'\n'
        f.write(line)
        
        for i in item:
            line = '\t'.join(i)+'\n'
            f.write(line)
        
        line = '\n'
        f.write(line)